<a href="https://colab.research.google.com/github/Ahauya/Data-Science-projects/blob/main/Sales_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter


# In[2]:


df = pd.read_csv("Sales_Data/Sales_April_2019.csv")

files = [ file for file in os.listdir("Sales_Data")]

all_months_data=pd.DataFrame()

for file in files:
    df = pd.read_csv("Sales_Data/"+ file)
    all_months_data = pd.concat([all_months_data, df])
    
all_months_data.to_csv("all_data.csv", index= False)


# In[3]:


all_data = pd.read_csv("all_data.csv")
all_data.head()


# In[4]:


## Clean the Data


# In[5]:


nan_df= all_data[all_data.isna().any(axis=1)]

nan_df.head()

all_data=all_data.dropna(how="all")
all_data.head()


# In[6]:


all_data = all_data[all_data ["Order Date"].str[0:2] != "Or"]


# In[7]:


##Convert Columns to correct data type


# In[8]:


all_data["Quantity Ordered"]=pd.to_numeric(all_data["Quantity Ordered"])
all_data["Price Each"] = pd.to_numeric (all_data["Price Each"])


# In[9]:


## Add Month column


# In[10]:


all_data["Month"]= all_data  ["Order Date"].str[0:2]
all_data["Month"]= all_data["Month"].astype("int32")
all_data.head()


# In[11]:


## Adding sales Column


# In[12]:


all_data["Sales"]= all_data["Quantity Ordered"] * all_data["Price Each"]
all_data.head()


# In[13]:


## Add a city Column to the data 


# In[25]:


def get_city(address):
    return address.split(",")[1]

def get_state(address):
    return address.split(",")[2].split(" ")[1]

all_data["City"] = all_data ["Purchase Address"].apply(lambda x: get_city(x) + ""+ get_state(x))

# all_data.drop(columns ="Column")

all_data.head(5)


# ## Best sales month And the Earnings of the Month

# In[35]:


results = all_data.groupby("Month").sum()


# In[36]:


months = range (1,13)
plt.bar(months,results["Sales"])
plt.xticks(months)
plt.ylabel("Sales in USD ($)")
plt.ylabel("Month Numbers")
plt.show()


# ## The City with highest Sales

# In[31]:


results = all_data.groupby("City").sum()
results


# In[33]:


cities = [city for city, df in all_data.groupby("City")]

plt.bar(cities,results["Sales"])
plt.xticks(cities, rotation ="vertical", size =8)
plt.ylabel("Sales in USD ($)")
plt.xlabel("City Name")
plt.show()


# ## When to use Marketing or Advertisement to maximaze Product sales

# In[38]:


all_data["Order Date"] = pd.to_datetime (all_data["Order Date"])


# In[40]:


all_data["Hour"] = all_data["Order Date"].dt.hour
all_data["Minute"] = all_data["Order Date"].dt.minute

all_data.head()


# In[44]:


hours = [hour for hour, df in all_data.groupby("Hour")]
plt.plot(hours, all_data.groupby (["Hour"]).count())

plt.xticks(hours)
plt.xlabel("Hour")
plt.ylabel ("Number Of Orders")
plt.grid()

plt.show()


# ## Product Mostly sold together

# In[49]:


df = all_data[all_data["Order ID"].duplicated(keep =False)]

df["Grouped"] = df.groupby("Order ID")["Product"].transform(lambda x: ",". join(x))

df = df [["Order ID", "Grouped"]].drop_duplicates()

df.head()


# In[54]:


count = Counter()

for row in df["Grouped"]:
    row_list = row.split(",")
    count.update(Counter(combinations(row_list,2)))

for key,value in count.most_common(10):
    print(key,value)


# ## product sold the most and why?

# In[62]:


product_group = all_data.groupby("Product")
quantity_ordered = product_group.sum()["Quantity Ordered"]

products = [ product for product, df in product_group]

plt.bar(products,quantity_ordered)
plt.ylabel("Quantity_Ordered")
plt.xlabel("Product")
plt.xticks(products, rotation ="vertical", size =8)
plt.show()


# In[72]:


prices =all_data.groupby("Product").mean()["Price Each"]

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(products, quantity_ordered, color="g")
ax2.plot(products, prices, 'b-')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity_Ordered', color='g')
ax2.set_ylabel('Price ($)', color='b')
ax1.set_xticklabels(products, rotation ="vertical", size =8)

plt.show()

